# **Defending the Edge: Evaluating OT Performance through Euclidean Measurements**

This algorithm presents the results from aggregating and measuring Euclideans between the OT, Defensive Rusher and Quarterback, see equation 1 below.  There are six aspects of this evaluation of defending the edge. Each aspect has it's own algorithm managing the aggregation of the data and metrics for it.


![euclidean distance eq1](https://github.com/autonomous019/NFL_Big_Data_Bowl_23/blob/main/euclidean_distance1.png?raw=true) (1)


**Six Aspects of Evaluation:** 
* Quality of Block Index (QBI)
* Block Efficiency
* Block Momentum
* QB-DE Rating
* QB-DE Euclidean
* Straight Lines Euclideans

**-Frame Metrics Aggregation:** gathers relevant x,y coordinates of QB, OT, and DE or other rusher, measures the required euclideans and writes these values to the 'frame_metrics' table in the db. <a href="https://colab.research.google.com/drive/1nPgD7U6zJIHLGlonTdbDhq3ApHCcMFCM?usp=sharing">OT_Defending_The_Edge_Metric_Aggregation.ipynb</a> notebook file. Frame metrics is in the frame_metrics table in the db. 

**-QBI: Quality Block Index** is a rating system that penalizes based on the OT either being beaten or allowing himself or the rusher to enter three zones around the QB: buffer (1.5yds), danger (1.0yds) or sack (<.7yds).  It is based on a scale of 0-3 and comprises the primary metric for overall evaluation akin to PFF ranking of OL. These are strictly speaking position based or coordinate based ratings relative to the QB. Qualitative metrics are noted below I use two scales the QBI and the QBI+Style Metrics for evaluation. QBI data is held in the qbi_metrics table in the db. 

**-Block Metrics:** Is an evaluation of such things as Momentum, the euclidean relative to start and end of play during interface between blocker and defender, the QB-DE rating and euclidean scores. So that 3 metrics are created through the block metrics algo: qb-de rating (0,1), qb-de euclidean (float), momentum (0-100) and efficiency (0-100). QB-DE Euclidean is the measurement of whether the distance has increased from the start of itnerface to the end of the interface b/w the Blocker and Defender, increase in distance of defender from QB. Momentum is a measure of whether the blocker slows the rate of closure on the QB, measured on a frame by frame basis, score reflects total percentage of frames where momentum decreased during interface. Efficiency is a measure of whether a block is using arm block or close or control block style (i.e. body close to other body). <a href="https://colab.research.google.com/drive/1sGJMSsW4gVAAPmYzmH0JQ_VeP-vQbQOp?usp=sharing">QBI and Block Metric Algorithm Notebook</a> Block Metrics data is held in the block_metrics table in the db. 

**-Straight Lines Divergence:** Is an evaluation based on the concept of divergence of the defender from taking a straight line toward the quarterback, the ability of the blocker to deter the rusher from the shortest path toward his target, the QB, it is also based in measuring the euclidean. This euclidean is a measurement from the defenders position compared to his predicted position on a straight line to the QB from his previous position. The higher the euclidean distance the better the result, the higher one is off the straight line to the QB. Straight Lines is held in the lines_metrics table in the db. 

**-Play and Game Results:** This current algorithm, displays the results on a play by play basis and game basis and presents final QBI Rating, used for a more objective evaluation and the QBI+Style (QBI+) evaluation which includes measures that are a question of blocking styles and more subjective but informative on an individual level of blocking style. The final QBI+ is on a scale of 0-5.5 with QBI 3pts and QB-DE Rating (.5pts), QB-DE Euclidean (.5pts), Lines Rating (.5pts), Momentum (.5pts) and Efficiency (.5pts). Scores are 0 or .5 based on a threshold value, see paper linked below for more information. 

**-Plotting:** this algorithm plots plays for the front 5 vs front 5 or for individual defender against blocker with reference to the QB. I have generated a library of 1 on 1 plots available at https://drive.google.com/drive/folders/1J-aUNbdA4JR2sLAj_nBjyn6IsjmnwznF?usp=sharing (currenlty comprised of over 9000 plots listed by nflId and gameId for each rated play like the inline plot below).  The <a href="https://colab.research.google.com/drive/1C6aM5tLTj6luCqj951OdMvQhbH_IHBQE?usp=sharing">plotting notebook</a> contains the code for generating plots.  A sample animated <a href="https://github.com/autonomous019/NFL_Big_Data_Bowl_23/blob/main/2021090900_480%20(4).mp4?raw=true">front 5 vs front 5 plot</a>.

![2236](https://github.com/autonomous019/NFL_Big_Data_Bowl_23/blob/main/2236.png?raw=true)


**Time Length:** It should be noted that there is a limit on the evaluation of plays, I do not evaluate past 4 seconds of a play, this effects a minority of the plays as the large majority of plays are less then 4 seconds. 1 second equals 1 frame (10fps) in the NFL data. Also, one can see a distinction in the length of plays, one could query the database based on frames (i.e. where total_frames = 25, etc) to see results for a given time range in the plays and compare the scores for different time lengths. 

Read More in Paper: Defending the Edge: Evaluating OT Performance through Euclidean Measurements, https://docs.google.com/document/d/e/2PACX-1vRQXvtUFV5mFOr2nH9DgBU9rVTpTOjoElIT6JAr8RX4WWCNpKVBmsyzJz-d1fYfzz09fuwPuGC3u943/pub


github repo: https://github.com/autonomous019/NFL_Big_Data_Bowl_23

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from functools import partial
from matplotlib.animation import writers
import os



#%matplotlib inline

In [ ]:
# Import from GoogleDrive
'''
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("//content/gdrive/MyDrive/nfl-big-data-bowl-2023/")

data_dir = "/content/gdrive/My Drive/nfl-big-data-bowl-2023/"
save_dir = "/content/gdrive/My Drive/nfl-big-data-bowl-2023/charts/"
lines_plots_dir = "/content/gdrive/My Drive/nfl-big-data-bowl-2023/lines_plots/"
#if on kaggle
#data_dir = "/kaggle/input/nfl-big-data-bowl-2023/"

'''
data_dir = "/kaggle/input/4-seconds/40_databowl.db"

OUTPUT_DIR = './'
lines_plots_dir = OUTPUT_DIR
save_dir = OUTPUT_DIR






In [ ]:
import sqlite3

con = sqlite3.connect(data_dir)



# **General Functions**

In [ ]:
def get_scores(nfl_id):
    sql = "select * from qbi_metrics where blocker_id = "+str(nfl_id)
    qbi = pd.read_sql_query(sql, con)
    sql = "select * from block_metrics where blocker_id = "+str(nfl_id)
    block = pd.read_sql_query(sql, con)
    sql = "select * from lines_metrics where blocker_id = "+str(nfl_id)
    lines = pd.read_sql_query(sql, con)

    #with pd.option_context('display.max_rows', None,):
    #    display(query)
    return qbi,block,lines


def get_scores_by_game(nfl_id, game_id):
    sql = "select * from qbi_metrics where blocker_id = "+str(nfl_id)+" AND gameId = "+str(game_id)
    qbi = pd.read_sql_query(sql, con)
    sql = "select * from block_metrics where blocker_id = "+str(nfl_id)+" AND gameId = "+str(game_id)
    block = pd.read_sql_query(sql, con)
    sql = "select * from lines_metrics where blocker_id = "+str(nfl_id)+" AND gameId = "+str(game_id)
    lines = pd.read_sql_query(sql, con)

    #with pd.option_context('display.max_rows', None,):
    #    display(query)
    return qbi,block,lines

def get_scores_by_play(nfl_id, game_id, play_id):
    sql = "select * from qbi_metrics where blocker_id = "+str(nfl_id)+" AND gameId = "+str(game_id)+" AND playId = "+str(play_id)
    qbi = pd.read_sql_query(sql, con)
    sql = "select * from block_metrics where blocker_id = "+str(nfl_id)+" AND gameId = "+str(game_id)+" AND playId = "+str(play_id)

    block = pd.read_sql_query(sql, con)
    sql = "select * from lines_metrics where blocker_id = "+str(nfl_id)+" AND gameId = "+str(game_id)+" AND playId = "+str(play_id)

    lines = pd.read_sql_query(sql, con)

    #with pd.option_context('display.max_rows', None,):
    #    display(query)
    return qbi,block,lines


def get_metric_nflids(): 
    sql = """
           select DISTINCT(reference1_id) from frame_metrics where ref1_role IN ('LT', 'RT') 
    """ 
    query = pd.read_sql_query(sql, con)
    #print(query)
    return query

def get_game_ids(nfl_id):
  sql = """
           SELECT DISTINCT(gameId) from block_metrics where blocker_id = """+str(nfl_id)
  
  query = pd.read_sql_query(sql, con)
  #print(query)
  return query


def get_play_ids(nfl_id, game_id):
  sql = """
           SELECT DISTINCT(playId) from block_metrics where blocker_id = """+str(nfl_id)+""" AND gameId = """+str(game_id)
  
  query = pd.read_sql_query(sql, con)
  #print(query)
  return query






# **Ranking Functions**

In [ ]:
'''
def players_ranking(nfl_id, field):
  sql = "select ld.rowid, ld.* from leaderboard as ld WHERE ld.blocker_id = "+str(nfl_id)+" ORDER BY ld."+field+" DESC"
  query = pd.read_sql_query(sql, con)
  return query

def leader_board(field):
  sql = "select ld.blocker_id, ld.name, ld.qbi_total_plays, ld.qbi_avg, ld.sack, ld.danger, ld.buffer, ld.regained, ld.beaten, ld.qbi_set, ld.bm_momentum_avg, ld.bm_efficiency_avg, ld.lines_rating_avg from leaderboard as ld WHERE ld.qbi_total_plays > 99 ORDER BY ld."+field+" DESC"
  query = pd.read_sql_query(sql, con)
  #display(query)
  return query

def get_name(nfl_id):
  sql = "select displayName, weight, height from players where nflId = "+str(nfl_id)
  query = pd.read_sql_query(sql, con)
  display(query)
  return query

def get_combine_info(name):
  name = name.replace("'", " ")
  sql = "select * from combine where Name LIKE '%"+name+"%'"
  query = pd.read_sql_query(sql, con)
  display(query)
  return query


def update_leaderboard(blocker_id, field, field_value):
  cursor = con.cursor()
  sql = "UPDATE leaderboard SET "+field+" = "+str(field_value)+" WHERE blocker_id = "+ str(blocker_id)
  print(sql)
  print()   
  # Execute the query
  cursor.execute(sql)
  # the connection is not autocommited by default. So we must commit to save our changes.
  con.commit()


'''

# **Add Player Metrics to Plays Results Table Functions**

In [ ]:
def Average(lst):
    return sum(lst) / len(lst)

def qbi_sackarea(nfl_id):
  sql = "SELECT count(rush_sackarea) as rush_sackarea from qbi_metrics where blocker_id = " + str(nfl_id) + " AND rush_sackarea = 1"
  query = pd.read_sql_query(sql, con)
  return query

def qbi_dangerarea(nfl_id):
  sql = "SELECT count(rush_dangerarea) as rush_dangerarea from qbi_metrics where blocker_id = " + str(nfl_id) + " AND rush_dangerarea = 1"
  query = pd.read_sql_query(sql, con)
  return query

def qbi_bufferarea(nfl_id):
  sql = "SELECT count(rush_buffer) as rush_buffer from qbi_metrics where blocker_id = " + str(nfl_id) + " AND rush_buffer = 1"
  query = pd.read_sql_query(sql, con)
  return query

def qbi_beaten(nfl_id):
  sql = "SELECT count(beaten) as beaten from qbi_metrics where blocker_id = " + str(nfl_id) + " AND beaten = 1 and regained=0"
  query = pd.read_sql_query(sql, con)
  return query

def qbi_regained(nfl_id):
  sql = "SELECT count(regained) as regained from qbi_metrics where blocker_id = " + str(nfl_id) + " AND regained = 1 and beaten=1"
  query = pd.read_sql_query(sql, con)
  return query

def blocks_data(nfl_id):
  sql = "SELECT * from block_metrics where blocker_id = " + str(nfl_id)
  query = pd.read_sql_query(sql, con)
  return query

def get_qbi_level_cnt(nfl_id):
  sql = "SELECT DISTINCT(qbi_rating) as qbi_rating, count(playId) as playId from qbi_metrics where blocker_id = "+ str(nfl_id)+" GROUP BY qbi_rating"
  query = pd.read_sql_query(sql, con)
  return query

def add_leader_to_board(blocker_id, name, qbi_total_plays, qbi_avg, sack, danger, buffer, regained, beaten, qbi_set, bm_momentum_avg, bm_efficiency_avg, bm_total_plays, bm_total_frames, bm_total_interface_frames, bm_control_frames, lines_rating_avg, lm_total_plays, qb_de_eucl_avg, qb_de_rating, weight, height, forty_speed, bench, broad, cone_speed, shuttle_speed):
  name = name.replace("'", " ")
  cursor = con.cursor()
  sql = """
      INSERT INTO `leaderboard` (`blocker_id`, `name`, `qbi_total_plays`, `qbi_avg`, `sack`, `danger`, `buffer`, `regained`, `beaten`, `qbi_set`, 
      `bm_momentum_avg`, `bm_efficiency_avg`, `bm_total_plays`, `bm_total_frames`, `bm_total_interface_frames`, `bm_control_frames`, 
      `lines_rating_avg`, `lm_total_plays`,  `qb_de_eucl_avg`, `qb_de_rating`, `weight`, `height`, `forty_speed`, `bench`, `broad`, `cone_speed`, `shuttle_speed` ) 
      VALUES (""" +str(blocker_id)+""", '""" +str(name)+"""', """ +str(qbi_total_plays)+""", """ +str(qbi_avg)+""", """ +str(sack)+""", """ +str(danger)+""", """ +str(buffer)+""", """ +str(regained)+""", """ +str(beaten)+""", '""" +str(qbi_set)+"""',     
      """ +str(bm_momentum_avg)+""", """ +str(bm_efficiency_avg)+""", """ +str(bm_total_plays)+""", """ +str(bm_total_frames)+""", """ +str(bm_total_interface_frames)+""", """ +str(bm_control_frames)+""", 
      """ +str(lines_rating_avg)+""", """ +str(lm_total_plays)+""",  """+str(qb_de_eucl_avg)+""", """+ str(qb_de_rating)+""",  
      """ +str(weight)+""", '""" +str(height)+"""', """ +str(forty_speed)+""", """ +str(bench)+""", """ +str(broad)+""", """ +str(cone_speed)+""", """ +str(shuttle_speed)+""" 
      )
  """
  print(sql)
     
  # Execute the query
  cursor.execute(sql)
  # the connection is not autocommited by default. So we must commit to save our changes.
  con.commit()
  sql2 = """
      INSERT INTO `ranks_to_leaderboard` (`blocker_id` ) 
      VALUES (""" +str(blocker_id)+""" )
  """
  print(sql2)

  # Execute the query
  cursor.execute(sql2)
  # the connection is not autocommited by default. So we must commit to save our changes.
  con.commit()

def get_leaderboard_nflids(): 
    sql = """
           select DISTINCT(blocker_id) from leaderboard where qbi_total_plays > 99 
    """ 
    query = pd.read_sql_query(sql, con)
    #print(query)
    return query

def get_leaderboard_info(nfl_id): 
    sql = """
           select * from leaderboard where qbi_total_plays > 99 AND blocker_id = """+str(nfl_id) 
     
    query = pd.read_sql_query(sql, con)
    #print(query)
    return query

def get_qbi_metrics(nfl_id, game_id, play_id):
  sql = "SELECT * from qbi_metrics where blocker_id = "+ str(nfl_id)+" AND gameId = "+str(game_id)+" AND playId = "+str(play_id) 
  query = pd.read_sql_query(sql, con)
  return query

def get_block_metrics(nfl_id, game_id, play_id):
  sql = "SELECT * from block_metrics where blocker_id = "+ str(nfl_id)+" AND gameId = "+str(game_id)+" AND playId = "+str(play_id) 
  query = pd.read_sql_query(sql, con)
  return query

def get_lines_metrics(nfl_id, game_id, play_id):
  sql = "SELECT * from lines_metrics where blocker_id = "+ str(nfl_id)+" AND gameId = "+str(game_id)+" AND playId = "+str(play_id) 
  query = pd.read_sql_query(sql, con)
  return query

def get_details(game_id, play_id):
  sql = "SELECT * from plays where gameId = "+str(game_id)+" AND playId = "+str(play_id) 
  query = pd.read_sql_query(sql, con)
  return query

def passer_for_play(nfl_id, game_id, play_id):

    sql = """
           select DISTINCT reference1_id from frame_metrics as fm WHERE fm.reference2_id = """ + str(nfl_id) + """ 
           AND fm.playId = """ + str(play_id) + """  
           AND fm.gameId = """ + str(game_id) + """
    """  
    query = pd.read_sql_query(sql, con)
    return query

def defender_for_play(nfl_id, game_id, play_id):

    sql = """
           select DISTINCT reference2_id from frame_metrics as fm WHERE fm.reference1_id = """ + str(nfl_id) + """ 
           AND fm.playId = """ + str(play_id) + """  
           AND fm.gameId = """ + str(game_id) + """
    """  
    query = pd.read_sql_query(sql, con)
    return query


# Python code to merge dict using update() method
def Merge(dict1, dict2):
    return(dict2.update(dict1))

def add_play_result(data):
  cursor = con.cursor()

  print(data)
  #c.execute("INSERT INTO users VALUES (:id, :name, :dob)", user1) 

  sql = """INSERT INTO `play_results` VALUES (:blocker_id, :defender_id, :name, :weight, :height, :qbi_rating, :t_sack, :t_danger, :t_buffer, :t_block_sack, :t_block_danger, 
  :t_block_buffer, :t_regained, :t_beaten, :int_cnt, :cntrl_cnt, :total_frames, :efficiency, 
  :momentum, :qb_de_eucl_avg, :qb_de_rating, :lines_score, :gameId, :playId)""" 

  
  #print(sql)
  #cursor.execute(sql)
  cursor.execute(sql, data)
  con.commit()







############################################### END FUNCTIONS ###########################



# **Aggregate Player Results Driver**

In [ ]:
'''
uncomment to aggregate player results to play_results table in db. 


nflids = get_leaderboard_nflids()
nfl_ids = nflids[['blocker_id']]
nflids_len = len(nfl_ids)
qbi_mis = []
games_list = []

for c,r in nfl_ids.iterrows():
  #if c > 0:
  #  break
  
  nfl_id = r['blocker_id']

  get_info = get_leaderboard_info(nfl_id)
  name = get_info['name'].values
  for n in name:
    name = n
  name = str(name)
  print(name)

  weight = get_info['weight'].values
  for n in weight:
    weight = n
  
  height = get_info['height'].values
  for n in height:
    height = n

  #get games
  games = get_game_ids(nfl_id)
  for c,g in games.iterrows():

    game_id = g['gameId'] 
    games_list.append(game_id) 
    plays = get_play_ids(nfl_id, game_id)
    for c,p in plays.iterrows():
      play_result = {}
      play_id = p['playId']
      print(name, nfl_id, game_id, play_id)
      #get info from plays
      details = get_details(game_id, play_id)
      qbi = get_qbi_metrics(nfl_id, game_id, play_id)
      block = get_block_metrics(nfl_id, game_id, play_id)
      lines = get_lines_metrics(nfl_id, game_id, play_id)

      #QBI 
      blocker_id = qbi['blocker_id'][0]
      defender_id = qbi['defender_id'][0]
      qbi_rating = qbi['qbi_rating'][0]
      t_block_sack = qbi['rush_sackarea'][0]
      t_block_danger = qbi['block_dangerarea'][0]
      t_block_buffer = qbi['block_buffer'][0]
      t_sack = qbi['rush_sackarea'][0]
      t_danger = qbi['rush_dangerarea'][0]
      t_buffer = qbi['rush_buffer'][0]     
      t_regained = qbi['regained'][0]
      t_beaten = qbi['beaten'][0] - t_regained
      #qbi_data = dict({ 'blocker_id': blocker_id, 'defender_id':defender_id, 'qbi_rating': qbi_rating, 't_sack': t_sack, 't_danger': t_danger, 't_buffer': t_buffer, 't_block_sack': t_block_sack, 't_block_danger': t_block_danger, 't_block_buffer': t_block_buffer, 't_regained': t_regained, 't_beaten': t_beaten})
      #print(qbi_data)
      
      #Blocks


      for c,b in block.iterrows():
        int_cnt = b['interface_cnt']
        cntrl_cnt = b['control_cnt']
        total_frames = b['total_frames']
        effic = cntrl_cnt/int_cnt
        momentum = b['momentum_rating']
        qb_de_eucl_avg = b['qb_de_eucl_avg']
        qb_de_rating = b['qb_de_rating']
        #block_data = ({'int_cnt':int_cnt, 'cntrl_cnt': cntrl_cnt, 'total_frames': total_frames, 'efficiency':effic, 'momentum':momentum, 'qb_de_eucl_avg':qb_de_eucl_avg, 'qb_de_rating':qb_de_rating})
        #print(block_data)

        #Lines
        if len(lines) == 0:
          lines_score = 0
        else:
          lines_score = lines['score'][0]

        #lines_data = ({ 'lines_score': lines_score})
        #print(lines_data)
        data = ({ 'blocker_id': int(blocker_id), 'defender_id':int(defender_id), 'name': name, 'weight':int(weight), 'height':height, 'qbi_rating': qbi_rating, 't_sack': int(t_sack), 
                 't_danger': int(t_danger), 't_buffer': int(t_buffer), 
                 't_block_sack': int(t_block_sack), 't_block_danger': int(t_block_danger), 't_block_buffer': int(t_block_buffer), 't_regained': int(t_regained), 
                 't_beaten': int(t_beaten),
                 'int_cnt':int_cnt, 'cntrl_cnt': cntrl_cnt, 'total_frames': total_frames, 'efficiency':effic, 'momentum':momentum, 
                 'qb_de_eucl_avg':qb_de_eucl_avg, 'qb_de_rating':qb_de_rating, 'lines_score': lines_score, 'gameId':int(game_id), 'playId': int(play_id)
                 })
        

        
        
        print()
        print()
        print()
        #write play results, uncomment below to add to db
        #add_play_result(data)


    print("_______________________")
    print()
  #write game results

print(qbi_mis)
'''

# **Generate Results by Game Functions**

In [ ]:
def get_results(nfl_id): 
    sql = """
           select * from play_results where blocker_id = """+str(nfl_id) 
     
    query = pd.read_sql_query(sql, con)
    #print(query)
    return query

def get_player_games(nfl_id): 
    sql = """
           select DISTINCT(gameId) from play_results where blocker_id = """+str(nfl_id) 
     
    query = pd.read_sql_query(sql, con)
    #print(query)
    return query

def get_results_by_game(nfl_id, game_id): 
    sql = """
           select * from play_results where blocker_id = """+str(nfl_id)+""" 
           AND gameId = """+str(game_id)
     
    query = pd.read_sql_query(sql, con)
    #print(query)
    return query

def get_play_info(game_id, play_id): 
    sql = """
           select * from plays where gameId = """+str(game_id)+ """ 
           AND playId = """+str(play_id) 
     
    query = pd.read_sql_query(sql, con)
    #print(query)
    return query

def get_defender_info(nfl_id): 
    sql = """
           select * from players where nflId = """+str(nfl_id)
     
    query = pd.read_sql_query(sql, con)
    #print(query)
    return query

def get_defender_role(nfl_id, game_id, play_id): 
    sql = """
           select ref2_role from frame_metrics where reference2_id = """+str(nfl_id)+""" 
           AND gameId = """+str(game_id)+""" AND playId = """ +str(play_id)
     
    query = pd.read_sql_query(sql, con)
    #print(query)
    return query

def add_game_result(data):
  cursor = con.cursor()

  print(data)


  sql = """
           INSERT INTO `game_results` VALUES (:blocker_id, :gameId, :total_plays, :t_qbi, :t_ln, :t_qbde_rating, :t_qbde_eucl, :t_effic, :t_momentum, :t_scores, 
:c_cnt, :i_cnt, :c_qbi, :i_qbi, :c_ln, :i_ln, :c_qbde_rating, :i_qbde_rating, :c_qbde_eucl, :i_qbde_eucl, :c_effic, :i_effic, 
:c_momentum, :i_momentum, :c_scores, :i_scores, :success_rate)
""" 

  
  #print(sql)
  #cursor.execute(sql)
  cursor.execute(sql, data)
  con.commit()
  
def update_play_results(blocker_id, game_id, play_id, field, field_value):
  cursor = con.cursor()
  #play_total
  sql = "UPDATE play_results SET "+field+" = "+str(field_value)+" WHERE blocker_id = "+ str(blocker_id)+" AND gameId = "+str(game_id)+" AND playId = "+str(play_id)
  print(sql)
  print()   
  # Execute the query
  cursor.execute(sql)
  # the connection is not autocommited by default. So we must commit to save our changes.
  con.commit()




# **Display Results by Play, with Game Results**

The following will display the results of several metrics, such as QBI, Qb-DE-rating, Qb-DE-euclidean, Straight Lines Ratings, and Momentum. Results are displayed for each play, as well as for each game. Data is also presented to compare along Pass completed vs incompleted results for the previously mentioned metrics. 



In [ ]:
#9834 rated plays, removed 108 1.09%, 99% aggregated successfully.
#update_play_results() needs uncommented and add_game_results() to write to db. 
nflids = get_leaderboard_nflids()
nfl_ids = nflids[['blocker_id']]
nflids_len = len(nfl_ids)
games_list = []
  
for c,r in nfl_ids.iterrows():

  if c < 41:
    continue
  nfl_id = r['blocker_id']

  games = get_player_games(nfl_id)

  total_games = len(games)
  for c,g in games.iterrows():
    game_id = g['gameId']
    print()
    print()
    print("######################################################################")
    print("######################################################################")
    print("######################################################################")
    print()
    print(c)
    print(game_id)
    print(nfl_id)
    game_results = get_results_by_game(nfl_id, game_id)
    qbi_game_avg = game_results[['qbi_rating']].mean()
    qbi_game_avg = qbi_game_avg.values
    for n in qbi_game_avg:
      qbi_avg = n
    print("QBI AVG FOR GAME: ", qbi_avg)

    qbi_list = []
    completed_qbi_list = []
    incompleted_qbi_list = []
    completed_lines_list = []
    incompleted_lines_list = []
    completed_qbde_rating_list = []
    incompleted_qbde_rating_list = []
    completed_qbde_eucl_list = []
    incompleted_qbde_eucl_list = []
    completed_efficiency_list = []
    incompleted_efficiency_list = []
    completed_momentum_list = []
    incompleted_momentum_list = []
    completed_play_scores = []
    incompleted_play_scores = []

    play_scores = []
    qbi_list = []
    lines_list = []
    qbde_rating_list = []
    qbde_eucl_list = []
    efficiency_list = []
    momentum_list = []


    for c,r in game_results.iterrows():

      blocker_id = r['blocker_id']
      name = r['name']
      game_id = r['gameId']
      play_id = r['playId']
      qbi_avg = r['qbi_rating']

      qb_de_eucl_avg = r['qb_de_eucl_avg']
      #if qb_de_eucl_avg > .4 add .50 to score dictionary
      if qb_de_eucl_avg >= 4.0:
        qb_de_eucl_score = .5
      else:
        qb_de_eucl_score = 0

      qb_de_rating = r['qb_de_rating']
      #if qb_de_eucl_rating = 1 add .50 to score dictionary
      if qb_de_rating == 1:
        qb_de_rating_score = .5
      else:
        qb_de_rating_score = 0
      
      d_sack = r['t_sack']
      d_danger = r['t_danger']
      d_buffer = r['t_buffer']
      b_sack = r['t_block_sack']
      b_danger = r['t_block_danger']
      b_buffer = r['t_block_buffer']
      regained = r['t_regained']
      beaten = r['t_beaten']
      momentum = r['momentum']
      #if momentum > .5 add .50 to score dictionary
      if momentum >= .5:
        momentum_score = .5
      else:
        momentum_score = 0

      efficiency = r['efficiency']
      #if momentum > .25 add .50 to score dictionary
      if efficiency >= .25:
        efficiency_score = .5
      else:
        efficiency_score = 0

      lines_score = r['lines_score']
      #if lines_score > .3 add .50 to score dictionary
      if lines_score >= .3:
        lines_score_score = .5
      else:
        lines_score_score = 0
      
      defender_id = r['defender_id']
      defender_name = get_defender_info(defender_id)
      for c,p in defender_name.iterrows():
        def_name = p['displayName']
      defender_role = get_defender_role(defender_id, game_id, play_id)
      play_score = qbi_avg + qb_de_eucl_score + qb_de_rating_score + momentum_score + efficiency_score + lines_score_score
      for c,p in defender_role.iterrows():
        def_role = p['ref2_role']
      int_cnt = r['int_cnt']
      cntrl_cnt = r['cntrl_cnt']
      total_frames = r['total_frames']
      play_info = get_play_info(game_id, play_id)
      for c,p in play_info.iterrows():
        desc = p['playDescription']
        complete = p['passResult']
        play_scores.append(play_score)
        qbi_list.append(qbi_avg)
        lines_list.append(lines_score)
        qbde_rating_list.append(qb_de_rating)
        qbde_eucl_list.append(qb_de_eucl_avg)
        efficiency_list.append(efficiency)
        momentum_list.append(momentum)

        if complete == 'C':
          completed_qbi_list.append(qbi_avg)
          completed_lines_list.append(lines_score)
          completed_qbde_rating_list.append(qb_de_rating)
          completed_qbde_eucl_list.append(qb_de_eucl_avg)
          completed_efficiency_list.append(efficiency)
          completed_play_scores.append(play_score)
          completed_momentum_list.append(momentum)

        else:
          incompleted_qbi_list.append(qbi_avg)
          incompleted_lines_list.append(lines_score)
          incompleted_qbde_rating_list.append(qb_de_rating)
          incompleted_qbde_eucl_list.append(qb_de_eucl_avg)
          incompleted_efficiency_list.append(efficiency)
          incompleted_play_scores.append(play_score)
          incompleted_momentum_list.append(momentum)
      
      #add qbi + qb_de_rating + qb_de_eucl + ln + momentum + effic = play total score
      # qb_de_eucl_score, qb_de_rating_score, momentum_score, efficiency_score, lines_score
      print()
      print("qb eucl score", qb_de_eucl_score, " qb_de_rating score: ", qb_de_rating_score, " momentum score: ", momentum_score, " effic scr: ", efficiency_score, " ln scr: ", lines_score_score)
      
      print("Play Score: ", play_score)
      print(play_id, " -- ", name, "QBI: ", qbi_avg, " ln: ", lines_score, " qb-de eucl: ", qb_de_eucl_avg, "qb-de rating: ", qb_de_rating)
      print("        ", "effic: ", efficiency, " mom: ", momentum)
      print(desc)
      print("Defender: ",def_name, " position: ", def_role)
      #update_play_results(blocker_id, game_id, play_id, 'play_total', play_score)

    print()
    print("completed qbi avg: ", Average(completed_qbi_list), len(completed_qbi_list))
    print("incompleted qbi avg: ", Average(incompleted_qbi_list), len(incompleted_qbi_list))
    print("completed lines avg: ", Average(completed_lines_list), len(completed_lines_list))
    print("incompleted lines avg: ", Average(incompleted_lines_list), len(incompleted_lines_list))
    print("completed qb-de eucl avg: ", Average(completed_qbde_rating_list), len(completed_qbde_rating_list))
    print("incompleted qb-de eucl avg: ", Average(incompleted_qbde_rating_list), len(incompleted_qbde_rating_list))
    print("completed effic avg: ", Average(completed_efficiency_list), len(completed_efficiency_list))
    print("incompleted effic avg: ", Average(incompleted_efficiency_list), len(incompleted_efficiency_list))
    print("completed momentum avg: ", Average(completed_momentum_list), len(completed_momentum_list))
    print("incompleted momentum avg: ", Average(incompleted_momentum_list), len(incompleted_momentum_list))
    print("completed play score avg: ", Average(completed_play_scores), len(completed_play_scores))
    print("incompleted play score avg: ", Average(incompleted_play_scores), len(incompleted_play_scores))
    print()
    success_plays = game_results[game_results['qbi_rating'] > 2.74]
    success_rate = len(success_plays) / len(game_results)
    print("Success plays: ", success_rate)

    c_qbi = Average(completed_qbi_list)
    i_qbi = Average(incompleted_qbi_list)
    c_ln = Average(completed_lines_list)
    i_ln = Average(incompleted_lines_list)
    c_qbde_rating = Average(completed_qbde_rating_list)
    i_qbde_rating = Average(incompleted_qbde_rating_list)
    c_qbde_eucl = Average(completed_qbde_eucl_list)
    i_qbde_eucl = Average(incompleted_qbde_eucl_list)
    c_effic = Average(completed_efficiency_list)
    i_effic = Average(incompleted_efficiency_list)
    c_momentum = Average(completed_momentum_list)
    i_momentum = Average(incompleted_momentum_list)
    c_scores = Average(completed_play_scores)
    i_scores = Average(incompleted_play_scores)
    #total scores
    t_qbi = Average(qbi_list)
    t_ln = Average(lines_list)
    t_qbde_rating = Average(qbde_rating_list)
    t_qbde_eucl = Average(qbde_eucl_list)
    t_effic = Average(efficiency_list)
    t_momentum = Average(momentum_list)
    t_scores = Average(play_scores)
    c_cnt = len(completed_qbi_list) 
    i_cnt = len(incompleted_qbi_list) 
    total_plays = len(play_scores)
    print("game results: ", t_qbi, t_ln, t_qbde_rating, t_qbde_eucl, t_effic, t_momentum, t_scores)    
    '''
    :blocker_id, :gameId, :total_plays, :t_qbi, :t_ln, :t_qbde_rating, :t_qbde_eucl, :t_effic, :t_momentum, :t_scores, 
:c_cnt, :i_cnt, :c_qbi, :i_qbi, :c_ln, :i_ln, :c_qbde_rating, :i_qbde_rating, :c_qbde_eucl, :i_qbde_eucl, :c_effic, :i_effic, 
:c_momentum, :i_momentum, :c_scores, :i_scores, :success_rate

    '''
    play_data = ({ 'blocker_id': int(blocker_id), 'gameId': int(game_id), 'total_plays': int(total_plays), 't_qbi': t_qbi, 't_ln':t_ln, 't_qbde_rating':t_qbde_rating,
               't_qbde_eucl': t_qbde_eucl, 't_effic':t_effic, 't_momentum':t_momentum, 't_scores':t_scores, 'c_cnt':int(c_cnt), 'i_cnt':int(i_cnt), 'c_qbi':c_qbi, 'i_qbi': i_qbi,
               'c_ln': c_ln, 'i_ln':i_ln, 'c_qbde_rating':c_qbde_rating, 'i_qbde_rating':i_qbde_rating, 'c_qbde_eucl':c_qbde_eucl, 'i_qbde_eucl':i_qbde_eucl,
               'c_effic':c_effic, 'i_effic':i_effic, 'c_momentum':c_momentum, 'i_momentum':i_momentum, 'c_scores':c_scores, 'i_scores':i_scores, 'success_rate':success_rate 
                 })

    #add_game_result(play_data)
    


    #write game results to weekly_results table
    
    print("------------------")
    print()
  print("Total games: ", total_games)
  print()
  print()
  print("=============================")  




# **Display Results: QBI plust Quality Scores** 

In [ ]:

sql = "select p.displayName, gr.blocker_id, AVG(gr.t_scores) from game_results as gr, players as p WHERE gr.blocker_id = p.nflId GROUP BY blocker_id ORDER BY AVG(gr.t_scores) DESC"
#sql = "select rowid, blocker_id from block_metrics"
query = pd.read_sql_query(sql, con)
print(len(query))
with pd.option_context('display.max_rows', None,):
  display(query)



TOP 5 Outside sources:

Rashawn Slater, predicted top LT for 2022, https://boltbeat.com/posts/pff-projects-chargers-rashawn-slater-2022

D.J. Humphries, 2022 Pro Bowl selection, https://www.revengeofthebirds.com/2022/1/31/22910421/arizona-cardinals-tackle-d-j-humphries-has-been-named-to-the-2022-pro-bowl-tyron-smith

Jake Matthews, top 10 entering 2020 season, https://www.atlantafalcons.com/news/early-bird-report-pff-names-jake-matthews-a-top-10-tackle-entering-season

Kolton Miller, 2nd Highest PFF rated tackle, https://www.si.com/nfl/raiders/news/las-vegas-raiders-maxx-crosby-kolton-miller-nate-hobbs-pff-top-101

Tristan Wirfs, top 3 Tackke, https://www.si.com/nfl/buccaneers/news/pro-football-focus-places-tristan-wirfs-into-an-elite-category



# # **Display Positional Based Quantitative Results based on QBI**

In [ ]:
sql = "select p.displayName, gr.blocker_id, AVG(gr.t_qbi) from game_results as gr, players as p WHERE gr.blocker_id = p.nflId GROUP BY blocker_id ORDER BY AVG(gr.t_qbi) DESC"
#sql = "select rowid, blocker_id from block_metrics"
query = pd.read_sql_query(sql, con)
print(len(query))
with pd.option_context('display.max_rows', None,):
  display(query)